<a href="https://colab.research.google.com/github/drehnstrom/spanner-examples/blob/main/colab-notebooks/Spanner_Generating_PK_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import uuid

print(uuid.uuid4())

print(str(uuid.uuid4()))

In [ ]:
def reverseBits(num,bitSize):
    # Convert number into binary representation
    # output will be like bin(10) = '0b10101'
    binary = bin(num)
    
    # Skip first two characters of binary
    # representation string and reverse
    # remaining string and then append zeros
    # after it. binary[-1:1:-1]  --> start
    # from last character and reverse it until
    # second last character from left
    reverse = binary[-1:1:-1]
    reverse = reverse + (bitSize - len(reverse))*'0'

    # converts reversed binary string into integer
    return  (int(reverse,2))  

print(reverseBits(1,32))
print(reverseBits(2,32))
print(reverseBits(10,32))
print(reverseBits(255,32))
print(reverseBits(1111,32))
print(reverseBits(2000,32))


In [ ]:
import binascii

text = str(1)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)


text = str(2)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(3)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(4)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(5)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(50)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(60)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(70)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

text = str(80)
encoded = binascii.crc32(text.encode('utf8'))
print(encoded)

